In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [6]:
import os
import pandas as pd
import numpy as np

In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [8]:
file_folder =  '../../data/input'
os.listdir(file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [9]:
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')

In [10]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left', left_on  = ['molecule_name', f'atom_index_{atom_idx}'], right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}', 'x': f'x_{atom_idx}', 'y': f'y_{atom_idx}', 'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)



test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

In [11]:
train_test[,typei:=.GRP, by="type"]

train_test[, N1 := length(unique(atom_index_1)) , by=c("molecule_name","atom_index_0")  ]
train_test[, N2 := length(unique(atom_index_0)) , by=c("molecule_name","atom_index_1")  ]

dt0 <- train_test[, paste(type,collapse=" "),
                  by=c("molecule_name","atom_index_0") ]
dt1 <- train_test[, paste(type,collapse=" "),
                  by=c("molecule_name","atom_index_1") ]
setnames(dt0,colnames(dt0), c("molecule_name","atom_index","V1") )
setnames(dt1,colnames(dt1), c("molecule_name","atom_index","V1") )

dt <- rbind( dt0, dt1  )
dt[, link := .GRP , by="V1" ]
dt[, N := .N, by="link"]
dt <- dt[, list(group=link[1]) , keyby="V1" ]
dt0[, link := dt[J(dt0$V1)]$group ]
dt1[, link := dt[J(dt1$V1)]$group ]
setkeyv(dt0, c("molecule_name","atom_index"))
setkeyv(dt1, c("molecule_name","atom_index"))

train_test[, link0 := dt0[J(train_test$molecule_name, train_test$atom_index_0)]$link ]
train_test[, link1 := dt1[J(train_test$molecule_name, train_test$atom_index_1)]$link ]
train_test[, linkN := .N, by=c("link0","link1") ]
head(train_test)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [12]:
train = reduce_mem_usage(train)

Mem. usage decreased to 408.70 Mb (54.0% reduction)


In [15]:
test = reduce_mem_usage(test)

Mem. usage decreased to 215.05 Mb (53.1% reduction)


In [18]:
train.drop(columns=['molecule_name','atom_index_0','atom_index_1','type','scalar_coupling_constant']).to_pickle('../../data/feats/molecular-properties-eda-and-models_train.pkl')

In [19]:
test.drop(columns=['molecule_name','atom_index_0','atom_index_1','type']).to_pickle('../../data/feats/molecular-properties-eda-and-models_test.pkl')